In [ ]:
# =============================================================================
# Main Script for CMG Parser _ ver 2.0, include parallel & grid output 
# Author: Honggeun Jo
# Date: 08/21/2023 - 
# =============================================================================

# =============================================================================
# Import librarays & Initializing
# =============================================================================
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import Subfunction as sub
import pyvista as pv

# Basic info for CMG
CMG =  '"C:\\Program Files\\CMG\\IMEX\\2022.10\\Win_x64\\EXE\\mx202210.exe"'
Current_Working_Dir = "C:/100 Research/cmg_parser"
os.chdir(Current_Working_Dir)
SIM =  "Forward_simulation.dat"
# Total_Number of realizations
No_realization = 10
# Reservoir extent
nx = 28; ny = 28; nz = 20
# number of wells 
No_well_Prod = 4
No_well_Inj = 1
# time steps
Ptime = np.arange(50, 1001, 50)
## Run CMG : subprocess.call([CMG,"-f",SIM])

In [ ]:
# ===a==========================================================================
# Load ensemble
# =============================================================================
Realizations_LogPerm = np.load('ensemble_101.npz')['logPermeability']
Realizations_Porosity = np.load('ensemble_101.npz')['porosity']

print(Realizations_LogPerm.shape, Realizations_Porosity.shape)

In [ ]:
# =============================================================================
# Make directories for ensemble
# =============================================================================
sub.Tstep(Ptime)
for i in range(No_realization) :
    if os.path.exists(f'realization_{str(i)}')==False: 
        os.mkdir(f'realization_{str(i)}')
    # main data file
    shutil.copy('Forward_simulation.dat', f'realization_{str(i)}/Forward_simulation.dat')
    # time steps
    shutil.copy('TSTEP.DAT', f'realization_{str(i)}/TSTEP.DAT')
    # conttol files
    shutil.copy('Control_Inj.DAT', f'realization_{str(i)}/Control_Inj.DAT')
    shutil.copy('Control_Prod.DAT', f'realization_{str(i)}/Control_Prod.DAT')
    # properties
    sub.SetPerm(Realizations_LogPerm[i].T, dir_ = f'realization_{str(i)}')
    sub.SetPoro(Realizations_Porosity[i].T, dir_ = f'realization_{str(i)}')
    

In [ ]:
# =============================================================================
# Generate variables to save results
# =============================================================================
WBHP, WGBP, WOPR, WWPR, WGPR, WCOP, WCWP, WCGP, WWIR, WCWI = sub.null_prod(No_realization, No_well_Prod, No_well_Inj, Ptime)
SO, SW, PRES = np.zeros((No_realization,len(Ptime)+1,nz,ny,nx)),np.zeros((No_realization,len(Ptime)+1,nz,ny,nx)),np.zeros((No_realization,len(Ptime)+1,nz,ny,nx))

In [ ]:
# =============================================================================
# Run simulations
# =============================================================================
for i in range(No_realization) :
    os.chdir(f'{Current_Working_Dir}/realization_'+str(i))
    #os.system(f'{CMG} -f {SIM}')
    WBHP[i], WGBP[i], WOPR[i], WWPR[i], WGPR[i], WCOP[i], WCWP[i], WCGP[i], WWIR[i], WCWI[i] = sub.read_out(Ptime, 'Forward_simulation.out', No_well_Prod, No_well_Inj)
    SO[i], SW[i], PRES[i] = sub.read_out_grid(Ptime,'Forward_simulation.out', nx, ny, nz)
    print(f'Ensemble simulation for realization_{str(i)} is completed')


In [ ]:
# WCOP:
plt.figure(figsize =(8,6))
for i_ in range(1,5):
    well_num = i_ -1
    plt.subplot(2,2,i_)
    plt.plot(Ptime,WCOP[i,:,well_num].T, '-', c = 'b', label = 'Inital ensemble',linewidth=0.5);
    for i in range(1,No_realization):      
        plt.plot(Ptime,WCOP[i,:,well_num].T, '-', c = 'b', linewidth=0.5);
    plt.legend()
    plt.title(f'Prod {i_}')
    plt.xlabel('Time, days')
    plt.ylabel('Production amount, MSTB')
    plt.grid('on')
plt.suptitle('Well Cumulative Oil Production')
plt.tight_layout()
plt.savefig('WCOP.svg')



In [ ]:
grid = pv.ImageData()
grid.dimensions = np.array(Realizations_LogPerm[0].T[:,::-1].shape) + 1
grid.origin = (0, 0, 0)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = Realizations_LogPerm[0].T[:,::-1].flatten(order="F")  # Flatten the array

plotter = pv.Plotter(window_size = (1500,1000), notebook=True)
plotter.add_text("3D visualization of Reservoir - Permeability of 1st realization\n", font_size=15)
# plotter.set_scale(zscale=0.75)
plotter.add_mesh(grid, show_edges=True)
plotter.remove_scalar_bar()
plotter.add_scalar_bar("Log permeability", vertical= True, position_y = 0.25, interactive=False)
plotter.update_scalar_bar_range([-1.7,4.7])
plotter.show_grid()
plotter.show(cpos =[(84.06758686482893, 85.59213397319502, 55.30007160018164),
            (14.0, 14.0, 10.0),
            (-0.2910466466174386, -0.29168853895593677, 0.9111584086943912)], 
            return_cpos=False)

In [ ]:
for i in range(len(Ptime)+1):
    grid = pv.ImageData()
    grid.dimensions = np.array(SO[0,i].T[:,::-1].shape) + 1
    grid.origin = (0, 0, 0)  # The bottom left corner of the data set
    grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
    grid.cell_data["values"] = SO[0,i].T[:,::-1].flatten(order="F")  # Flatten the array

    plotter = pv.Plotter(window_size = (1500,1000), notebook=True)
    plotter.add_text("3D visualization of Reservoir - Oil Saturation of 1st realization\n", font_size=15)
    # plotter.set_scale(zscale=0.75)
    plotter.add_mesh(grid, show_edges=True,cmap = 'jet')
    plotter.remove_scalar_bar()
    plotter.add_scalar_bar("Oil Sat.", vertical= True, position_y = 0.25, interactive=False)
    plotter.update_scalar_bar_range([0.177,0.84])
    plotter.show_grid()
    plotter.show(cpos =[(84.06758686482893, 85.59213397319502, 55.30007160018164),
                (14.0, 14.0, 10.0),
                (-0.2910466466174386, -0.29168853895593677, 0.9111584086943912)], 
                return_cpos=False)
    plotter.screenshot(filename = f'screenshot_{i}.png', transparent_background = True)
    plotter.close()

In [ ]:
from PIL import Image, ImageDraw
image_list = [Image.open(file) for file in [f'screenshot_{i}.png' for i in range(len(Ptime)+1)]]

image_list[0].save(
            'animation_v2.gif',
            save_all=True,
            append_images=image_list[1:], # append rest of the images
            duration=300, # in milliseconds
            loop=0)